<a href="https://colab.research.google.com/github/MCn21thCntry/Practical-Machine-Learning---from-the-rooter-to-the-tooter/blob/main/Homework_for_Module_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework for Module 4: Model Validation and Evaluation¶

Instructions:

This homework is designed to reinforce the concepts and techniques covered in Module 4. Please submit your answers and code (if applicable) in a Colab notebook.

https://www.kaggle.com/code/khaledhijja/module-4-ml-validation-and-evaluation

# Part 1: Regression Model Validation - Predicting Lot Area

For this part, we will use the same Iowa housing dataset, but this time, we will try to predict the LotArea (Lot size in square feet) of houses using a Decision Tree Regressor.

1. Prepare Data:

  * Load the Iowa dataset as you did in the module.
  * Select LotArea as the target variable (y).
  * Use the same numeric features (X) you used in the module (excluding 'SalePrice' and now also 'LotArea' if it's included, but it shouldn't be if you followed the module steps correctly).
  * Split the data into training and validation sets using train_test_split with test_size=0.2 and random_state=0.

In [30]:
## Reach the data in your drive
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Mount your Google Drive
drive.mount('/content/drive')

#Load the Iowa dataset as you did in the module.
# Load data from a CSV file in your Drive
iowa_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Iowa Housing Dataset/train.csv')
iowa_data.head()

## Handle missing values
for col in iowa_data.columns:
  if iowa_data[col].isna().sum() > 0:
    if iowa_data[col].dtype == 'object':
      iowa_data[col].fillna(iowa_data[col].mode()[0], inplace=True)
    else:
      iowa_data[col].fillna(iowa_data[col].mean(), inplace=True)

## handle the categorical features otherwiese they can cause errors during the model generation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Get the columns with 'object' data type.
object_columns = iowa_data.select_dtypes(include=['object']).columns # 43
for col in iowa_data.select_dtypes(include=["object"]).columns:
  ## Drop categoric features having only 1 class
  if iowa_data[col].nunique() < 2:
    iowa_data.drop(col, axis=1, inplace=True)
  ## Label encoding for feautures with 2 classses
  elif iowa_data[col].nunique() == 2:
    label_encoder = LabelEncoder()
    iowa_data[col] = label_encoder.fit_transform(iowa_data[col])
  ## One-hot encoding for features with more than 2 classes
  else:
    one_hot_encoder = OneHotEncoder(drop="first", max_categories=10, handle_unknown='infrequent_if_exist', min_frequency=0.01, sparse_output=False)
    iowa_data[col] = one_hot_encoder.fit_transform(iowa_data[[col]])

# Select LotArea as the target variable (y).
y = iowa_data['Lot Area']
# Use the same numeric features (X) you used in the module (excluding 'SalePrice' and now also 'LotArea' if it's included, but it shouldn't be if you followed the module steps correctly).
X = iowa_data.drop(['SalePrice', 'Lot Area'], axis=1)

# Split the data into training and validation sets using train_test_split with test_size=0.2 and random_state=0.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-30-56df99624ae4>:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  iowa_data[col].fillna(iowa_data[col].mean(), inplace=True)
<ipython-input-30-56df99624ae4>:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

2. Train and Evaluate a Decision Tree Regressor:

  * Train a DecisionTreeRegressor model (with random_state=0) on the training data (X_train, y_train).
  * Make predictions on the validation set (X_val).
  * Calculate the Mean Absolute Error (MAE) and Mean Squared * Error (MSE) on the validation set.
  * Print the MAE and MSE values.

In [33]:
# Train a DecisionTreeRegressor model (with random_state=0) on the training data (X_train, y_train).
tree_model = DecisionTreeRegressor(random_state=0)
tree_model.fit(X_train, y_train)
# Make predictions on the validation set (X_val).
predictions = tree_model.predict(X_val)

# Calculate the Mean Absolute Error (MAE) and Mean Squared * Error (MSE) on the validation set.
mae = mean_absolute_error(y_val, predictions)
mse = mean_squared_error(y_val, predictions)

# Print the MAE and MSE values.
print("After handling missing: Mean Absolute Error (MAE):", mae) # before handling missing: 4085.6704545454545
print("After handling missing: Mean Squared Error (MSE):", mse) # before handling missing: 272882503.37045455
y_val.describe()

After handling missing: Mean Absolute Error (MAE): 3167.1295454545457
After handling missing: Mean Squared Error (MSE): 129027281.65227273


,Lot Area
count,440.000000
mean,10089.527273
std,7061.013771
min,1476.000000
25%,7221.250000
50%,9372.500000
75%,11645.000000
max,115149.000000


3. Hyperparameter Tuning for Decision Tree:
  * Experiment with different values of the max_leaf_nodes hyperparameter for the DecisionTreeRegressor. Try at least 3 different values (e.g., 10, 100, 1000).
  * For each max_leaf_nodes value, train a new Decision Tree model and calculate the MAE on the validation set.
  * Record the MAE for each max_leaf_nodes value.
  * Which max_leaf_nodes value gives the lowest MAE on the validation set?

In [32]:
# Experiment with different values of the max_leaf_nodes hyperparameter for the DecisionTreeRegressor. Try at least 3 different values (e.g., 10, 100, 1000).
max_leaf_nodes_values = [10, 100, 1000]

# For each max_leaf_nodes value, train a new Decision Tree model and calculate the MAE on the validation set.
mae_list = []
for max_leaf_nodes in max_leaf_nodes_values:
    tree_model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    tree_model.fit(X_train, y_train)
    predictions = tree_model.predict(X_val)
    mae = mean_absolute_error(y_val, predictions)
    # Record the MAE for each max
    mae_list.append(mae)
# Which max_leaf_nodes value gives the lowest MAE on the validation set?
best_max_leaf_nodes = max_leaf_nodes_values[np.argmin(mae_list)]
print("After handling missing:: Best max_leaf_nodes value:", best_max_leaf_nodes) # before handling missing: 100

After handling missing:: Best max_leaf_nodes value: 100


4. Analysis (Written Answers):

  * Report the MAE and MSE values for the Decision Tree model (without hyperparameter tuning). What do these metrics tell you about the model's performance in predicting Lot Area?
    * MSE value looks so huge and MAE value looks good when we compare them with std and mean of y_val. These two metrics say that model performance is not perfect but good to take into account and use the model for new/unseen data so it is reliable to use for further.
  * Which max_leaf_nodes value (from your experiments) resulted in the lowest validation MAE? How did changing max_leaf_nodes affect the MAE?
    * 100 max_leaf_nodes value resulted in the lowest validation MAE. Number of max_leaf_nodes refers to the number of braches so it is related to splitting of features and it means that diversity and generalization the model includes. Providing these possessings, changing max_leaf_nodes affected the MAE.
  * Based on your experiments, what would be your chosen Decision Tree model (with what max_leaf_nodes value) for predicting Lot Area? Justify your choice.
    * I exactly would choose 100 max_leaf_nodes for predicting Lot Area because it makes the result the most successful as we saw in the example and compatible with the number of observations and features in terms of providing this kind of diversities.

## Part 2: Classification Model Validation - Predicting Above Average Lot Area

Now, let's switch to a classification task. We will try to predict whether a house has an "Above Average Lot Area" or "Below Average Lot Area".

1. Create a Binary Target Variable:

  * Calculate the median LotArea from the training set (y_train from Part 1).
  * Create a binary target variable y_binary_train_lot and y_binary_val_lot:
      * If a house's LotArea is above the median, set the binary target to 1 (for "Above Average").
      * Otherwise, set it to 0 (for "Below Average").

In [26]:
# Calculate the median LotArea from the training set (y_train from Part 1).
median_lot_area = y_train.median()

# Create a binary target variable y_binary_train_lot and y_binary_val_lot:
# If a house's LotArea is above the median, set the binary target to 1 (for "Above Average").
# Otherwise, set it to 0 (for "Below Average").
y_binary_train_lot = (y_train > median_lot_area).astype(int)
y_binary_val_lot = (y_val > median_lot_area).astype(int)
"""
(y_train > median_lot_area) creates a boolean Series where True indicates values above the median.
astype(int) converts the boolean values to integers (0 and 1).
"""

'\n(y_train > median_lot_area) creates a boolean Series where True indicates values above the median.\nastype(int) converts the boolean values to integers (0 and 1).\n'

2. Train and Evaluate a Decision Tree Classifier:
  * Train a DecisionTreeClassifier model (with random_state=0) on the training data (X_train, y_binary_train_lot).
  * Make predictions on the validation set (X_val).
  * Calculate the following classification metrics on the validation set:
      * Accuracy
      * Precision
      * Recall
      * Specificity
      * Confusion Matrix (display the matrix)
  * Print all the calculated metrics and display the confusion matrix.

In [34]:
# Train a DecisionTreeClassifier model (with random_state=0) on the training data (X_train, y_binary_train_lot).
from sklearn.tree import DecisionTreeClassifier
tree_model = DecisionTreeClassifier(random_state=0)
tree_model.fit(X_train, y_binary_train_lot)

# Make predictions on the validation set (X_val).
predictions = tree_model.predict(X_val)

# Calculate the following classification metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

accuracy = accuracy_score(y_binary_val_lot, predictions)
precision = precision_score(y_binary_val_lot, predictions)
recall = recall_score(y_binary_val_lot, predictions)

confusion = confusion_matrix(y_binary_val_lot, predictions)
# Calculate specificity
tn, fp, fn, tp = confusion.ravel()
specificity = tn / (tn + fp)

# Print all the calculated metrics and display the confusion matrix.
print("Accuracy:", accuracy) # before handling missing: 0.6977272727272728
print("Precision:", precision) # 0.6742081447963801
print("Recall:", recall) # 0.7095238095238096
print("Specificity:", specificity) # 0.6869565217391305
print("Confusion Matrix:")
print(confusion) # [[158  72] [ 61 149]]
print(tn, fp, fn, tp)

Accuracy: 0.7522727272727273
Precision: 0.730593607305936
Recall: 0.7619047619047619
Specificity: 0.7434782608695653
Confusion Matrix:
[[171  59]
 [ 50 160]]
171 59 50 160


3. Analysis (Written Answers):
  * Analyze the classification metrics you calculated for the Decision Tree Classifier. How would you describe the performance of the classifier in predicting "Above Average Lot Area"?
    * All the metrics are nearly in the same range and so close to each other and they are successful results. It demonstrates that the model designed with well-balanced.
  * Examine the Confusion Matrix. Are there more False Positives or False Negatives? What does this tell you about the types of errors the classifier is making?
    * False Positives are more than False Negatives. I tells me lower than average predictions in Lot Area above average are more than higher than average in Lot Area below average.
  * In the context of predicting "Above Average Lot Area," if you wanted to prioritize minimizing False Positives (i.e., be very sure when you predict "Above Average"), which metric would you focus on, and why?
    * Exactly I would focus on precision metric because it is based on investigation of False Positives rate prediction, which means that if precision is high False Positives rate are low otherwise hig.